In [2]:
import time
import random
import json
import requests as rq
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import tls_client
from seleniumbase import SB

### Get raw data we want to find based on ###
***Step to do before***
- run "Load_json(any)" cell in <u>clean.ipynb</u> to get "raw_data.csv"
- then, run all cell below "Preparing data for visualization"

In [ ]:
init_data = pd.read_csv("data/raw_data.csv")
viz_data = pd.read_parquet("data/viz_data.parquet.gzip")

,abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:given-name,abstracts-retrieval-response.item.bibrecord.head.correspondence.person.ce:surname,eid,journal_title,citedby_count,title,doi,is_funding,issn,funding_agency_country,abstracts-retrieval-response.item.bibrecord.head.related-item.source.@country,abstracts-retrieval-response.item.bibrecord.head.related-item.source.sourcetitle,abstracts-retrieval-response.item.bibrecord.head.related-item.source.publisher.affiliation.@country,references,ASJC_code,authors,main_author,affiliations_country,ref_count,delivered_date
0,Krit,Pongpirul,2-s2.0-85077976956,"Radiology in Global Health: Strategies, Implem...",1,Public health and international epidemiology f...,10.1007/978-3-319-98485-8_15,0,None,None,None,None,None,[Winslow CE. The untilled fields of public hea...,[2700],"[Pongpirul K., Lungren M.P.]",Pongpirul K.,"[Baltimore, United States, Stanford, United St...",76,2020-05-19
1,None,None,2-s2.0-85060936020,Progress in Electromagnetics Research Symposium,1,Flexible Printed Active Antenna for Digital Te...,10.23919/PIERS.2018.8597669,1,19317360 15599450,None,None,None,None,"[Pratumsiri, T. And P. Janpugdee, ""Development...","[2208, 2504]","[Pratumsiri T., Janpugdee P.]",Pratumsiri T.,None,4,2020-02-10
2,Benjapon,Chalermsinsuwan,2-s2.0-85052201238,Chemical Engineering Science,21,Parametric study of hydrogen production via so...,10.1016/j.ces.2018.08.042,1,00092509,None,None,None,None,"[Abanades, J.C., Anthony, E.J., Lu, D.Y., Salv...","[1600, 1500, 2209]","[Phuakpunk K., Chalermsinsuwan B., Putivisutis...",Phuakpunk K.,None,42,2021-02-23
3,Suwadee,Kongparakul,2-s2.0-85051498032,Applied Surface Science,37,Superhydrophobic coating from fluoroalkylsilan...,10.1016/j.apsusc.2018.08.059,1,01694332,None,None,None,None,"[Abadi, S.R.H., Sebzari, M.R., Hemati, M., Rek...","[1600, 3104, 3100, 3110, 2508]","[Saengkaew J., Le D., Samart C., Sawada H., Ni...",Saengkaew J.,"[Nagoya, Japan, Pathum Thani, Thailand, Bangko...",45,2021-07-31
4,Charles S.,Henry,2-s2.0-85050678366,Analytica Chimica Acta,68,Electrochemical impedance-based DNA sensor usi...,10.1016/j.aca.2018.07.045,1,18734324 00032670,None,None,None,None,"[Davies, P.D.O., Pai, M., The diagnosis and mi...","[1602, 1303, 2304, 1607]","[Teengam P., Siangproh W., Tuantranont A., Vil...",Teengam P.,"[Pathum Thani, Thailand, Fort Collins, United ...",55,2020-02-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20211,Duy,Le,2-s2.0-85111945558,Catalysis Today,3,Long-chain bio-olefins production via oxidativ...,10.1016/j.cattod.2021.07.034,1,09205861,None,None,None,None,"[Zhao, D., Wang, X., Miller, J.B., Huber, G.W....","[1503, 1600]","[Le D., Chaidherasuwet N., Rueangthaweep A., K...",Le D.,"[Bangkok, Thailand]",63,2023-07-04
20212,None,None,2-s2.0-85111408415,Critical Reviews in Analytical Chemistry,11,Recent Developments and Applications of Microf...,10.1080/10408347.2021.1949695,1,15476510 10408347,http://sws.geonames.org/1605651/,None,None,None,[The Food and Agriculture Organization of the ...,[1602],"[Alahmad W., Varanusupakul P., Varanusupakul P.]",Alahmad W.,None,115,2023-02-21
20213,Tejendra,Pherali,2-s2.0-85110903700,Compare,5,"Social justice, education and peacebuilding: c...",10.1080/03057925.2021.1951666,1,14693623 03057925,None,None,None,None,"[Abuza, Z. 2009. “ The Rehabilitation of Jemaa...",[3304],[Pherali T.],Pherali T.,"[London, United Kingdom, Bangkok, Thailand]",76,2023-06-07
20214,Ratchaneegorn,Mapanao,2-s2.0-85106740832,Journal of Applied Aquaculture,6,Effects of black soldier fly (Hermetia illucen...,10.1080/10454438.2021.1923609,1,15450805 10454438,http://sws.geonames.org/1605651/,None,None,None,"[Ali, M., M. Zaher, M. Alam, and M. Hussain. 2...","[2303, 1104]","[Mapanao R., Jiwyam W., Nithikulworawong N., W...",Mapanao R.,"[Khon Kaen, Thailand, Bangkok, Thailand]",44,2023-03-04


Websites used

- https://www.researchgate.net for [references, citations]
  1. go to "/search"
  2. input using "{journal title} OR {author}"
  3. look for matching, click it
  4. find desired content

- https://service.elsevier.com for [subject code]
  1. get html content from "/app/answers/detail/a_id/15181/supporthub/scopus/"


### Find missing journal stats


Making search_key for website search bar (researchgate.net)


In [ ]:
pd.set_option("display.max_colwidth", None)

# missing references or citations
ref_count = "abstracts-retrieval-response.item.bibrecord.tail.bibliography.@refcount"
cit_count = "abstracts-retrieval-response.coredata.citedby-count"
title = "abstracts-retrieval-response.coredata.dc:title"
author = "abstracts-retrieval-response.coredata.dc:creator.author"
doi = "abstracts-retrieval-response.coredata.prism:doi"

missing = init_data[init_data[ref_count].isna() | init_data[cit_count].isna()]
sample = missing[[title, author, doi]]


# making search key
def extract_author_name(dict_str):
    res = eval(dict_str)
    name = res[0].get("preferred-name")
    return name.get("ce:surname", "") + " " + name.get("ce:given-name", "")

sample[author] = sample[author].apply(lambda x: extract_author_name(x))
sample["search_key"] = sample[author] + " OR " + '"' + sample[title] + '"'


# convert to list for scraping
sample[doi] = sample[doi].fillna("")
search_keys = sample["search_key"].to_list()
titles = sample[title].to_list()
dois = sample[doi].to_list()
rows = sample.shape[0]

In [ ]:
agent_list = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.2420.81",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14.4; rv:124.0) Gecko/20100101 Firefox/124.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.4.1 Safari/605.1.15",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 14_4_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 OPR/109.0.0.0",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux i686; rv:124.0) Gecko/20100101 Firefox/124.0",
]

def get_journal_links(dois , titles , search_keys):
    #find most likely journal link from provided parameters
    urls = []
    for i in range(len(titles)):
        time.sleep(random.uniform(1,3))
        
        # html from request
        header = {"User-Agent": agent_list[i % 10]}
        session = tls_client.Session(
            client_identifier="chrome112", random_tls_extension_order=True
        )
        res = session.get(
            f"https://www.researchgate.net/search/publication?q={search_keys[i]}",
            headers=header,
        )
        session.close()

        search_result_html = BeautifulSoup(res.content, "lxml")
        searched_title = search_result_html.select(
            "div.nova-legacy-v-publication-item__title a"
        )

        # no search result found
        if len(searched_title) == 0:
            urls.append("")
            continue

        idx = -1
        if dois[i] != "":
        # match by DOI
            meta_datas = search_result_html.select(
                "ul.nova-legacy-v-publication-item__meta-data li:nth-child(2) span"
            )

            #all DOIs from search results
            searched_doi = [(i.text.split(" ")[1]) for i in meta_datas]
            try:
                #check if one match
                idx = searched_doi.index(dois[i])
            except:
                #impossible to find, skip
                urls.append("")
                continue
        else:
        # match by title
            #all titles from search results
            searched_title_text = [i.text for i in searched_title]
            #look for similarity
            for j in range(len(searched_title_text)):
                if titles[i] in searched_title_text[j]:
                    idx = j
                    break

        if idx==-1:
            urls.append("")
            continue
        
        # get link from matched result
        href = searched_title[idx].get_attribute_list("href", "")
        
        print(f"{i}: {href[0]}")
        urls.append(href[0])
    
    return urls

def get_content(urls):
    ref_count = []
    cite_count = []

    for i in range(len(urls)):
        
        print(f"{(i/rows)*100.0:.2f}%...")
        
        # skip if don't have link
        if urls[i] == "":
            ref_count.append(np.nan)
            cite_count.append(np.nan)
            print(":Url not provided:")
            continue

        with SB(uc=True) as sb:
            sb.uc_open_with_reconnect(
                f"https://www.researchgate.net/{urls[i]}",
                3,
            )
            time.sleep(2)
            # dealing with CAPTCHA
            try:
                if sb.is_element_visible('input[value*="Verify"]'):
                    sb.uc_click('input[value*="Verify"]')
                else:
                    sb.uc_gui_click_captcha()
            except Exception:
                print("No captcha appeared")

            try:
                result = sb.find_elements(
                    By.CSS_SELECTOR,
                    "div.publication-details__section h2.nova-legacy-e-text.nova-legacy-e-text--size-m.nova-legacy-e-text--family-display.nova-legacy-e-text--spacing-none.nova-legacy-e-text--color-inherit",
                )
                cite_text = result[0].text
                ref_text = result[1].text
                refc = int(ref_text[ref_text.find("(") + 1 : ref_text.rfind(")")])
                citec = int(cite_text[cite_text.find("(") + 1 : cite_text.rfind(")")])

                ref_count.append(refc)
                cite_count.append(citec)
            except:
                ref_count.append(np.nan)
                cite_count.append(np.nan)
                print(":Failed to gathering:")
    return ref_count,cite_count



Scrape and save to csv

In [ ]:
links = get_journal_links(dois=dois , titles=titles , search_keys=search_keys)

In [ ]:
link_df = pd.DataFrame(links,columns=["href_to_journal"])
link_df.to_csv("data/ref_cite_count_href.csv")

In [ ]:
ref_c , cite_c = get_content(links)

In [ ]:
#convert back to dataframe with key and export
count_df = pd.DataFrame({"references_count":ref_c , "citations_count":cite_c , "title":missing[title] , "author":missing[author] , "doi":missing[doi]})
count_df.to_csv("data/ref_cite_count.csv")
count_df

### Get journal subject code from Elsevier 

In [ ]:
url = "https://service.elsevier.com/app/answers/detail/a_id/15181/supporthub/scopus/"
response = rq.get(url)
html = BeautifulSoup(response.content, "lxml")

header = html.select("table.order-table thead tr th")
body = html.select("table.order-table tbody tr")

label = [i.select_one("h3").text for i in header]
data = [[j.text for j in i.select("td p")] for i in body]

code_df = pd.DataFrame(data, columns=label)
code_df.to_csv("data/ASJC_cat.csv")
code_df[code_df.Code == "2105"]


### Get Coordinates from Country name using 'Geocode HERE' API
Country of affiliations with that journal

In [ ]:
base_url = "https://geocode.search.hereapi.com/v1/geocode"
api_keys = ["aByISMRAsDgzQPiDivruVUOqEpOMkvIIL1y84zFrHYY",
            "sDKAr-8Hlfdofp6B7eQjh7v-MwramnYklOKoQS07kr8",
            "bcF0z35tDTkWHDFWy7nNyErulWmC8VcUtXpWsIPRUvQ",
            "nsE1wCRkWGMcIoOmPbDoqq4E6nsSeN7LhMFSXYBNESQ",
            "71RYrr7MfJRMeG73SKNkhixJ4mZ5yLBP6LqOY26PFps"
            ]

#get all country list
country_exploded = viz_data.explode("affiliations_country")
country_list = country_exploded["affiliations_country"].unique()
country_list = country_list[np.where(country_list != None)]
lng = []
lat = []

key_idx = 0
for country in country_list:
    if country is None:
        #skip nan value
        continue
    
    session = tls_client.Session()
    response = session.get(base_url , params = {"q":country,"apiKey":api_keys[key_idx]} )
    data = json.loads(response.content)
    print(data)

    if data.get("items",-1) == -1:
        print("Fetch failed, trying another API-KEY...")
        key_idx = (key_idx+1) % len(api_keys)
        response = session.get(base_url , params = {"q":country,"apiKey":api_keys[key_idx]} )
        data = json.loads(response.content)
        
    if len(data.get("items")) == 0:
        # no item return, try only country
        response = session.get(base_url , params = {"q":country.split(",")[1],"apiKey":api_keys[key_idx]} )
        data = json.loads(response.content)
    
    coord = data["items"][0]["position"]
    lng.append(coord["lng"])
    lat.append(coord["lat"])
    session.close()
    
    time.sleep(0.1)

In [ ]:
coord_df = pd.DataFrame({"country_name":country_list , "latitude": lat, "longitude":lng })
coord_df.to_csv('data/coordinate_country.csv')